In [9]:
import pandas as pd
import os
import glob
import numpy as np
from functools import reduce

In [10]:
os.chdir('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/curated_datasets')
path = os.getcwd()
path

'/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/curated_datasets'

### Clean the revolution dataset

In [11]:
country_code = pd.read_csv(os.path.join('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3',
                                       "Country_code.csv"))
country_code.head()

,country_code,country_name
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AND,Andorra
4,AGO,Angola


In [12]:
revo_raw = pd.read_csv(os.path.join(path, "Revolution Final.csv"), usecols = [ 2, 3, 4, 5], dtype = {"year" : 'Int64'})
col_names = revo_raw.columns.to_list()
revo_raw.head()

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
4,10,2012,True,EGY


In [13]:
revo_raw.type_of_change.unique()

array([ 2,  7,  8, 10,  0,  9,  5, 11])

In [14]:
revo_raw[revo_raw["country_code"] == "DEU"]

,type_of_change,year,Revolution,country_code


In [15]:
col_names[3:0:-2]+ col_names[0:4:2]

['country_code', 'year', 'type_of_change', 'Revolution']

In [16]:
rows_list = []

year_serie= pd.Series(range(2010,2021)).to_list()

my_codes = country_code.country_code.to_list()

for year in year_serie:
    
    for code in my_codes:

        my_dict = {}
        my_dict[year] = code
        my_dict.update(my_dict)
        
        rows_list.append(my_dict)
        


In [17]:
year_code_df = pd.DataFrame(rows_list).stack().to_frame().reset_index()
year_code_df.rename(columns = {"level_1" : "year",
                               0 : "country_code"}, inplace = True)
year_code_df

,level_0,year,country_code
0,0,2010,AFG
1,1,2010,ALB
2,2,2010,DZA
3,3,2010,AND
4,4,2010,AGO
...,...,...,...
2140,2140,2020,VEN
2141,2141,2020,VNM
2142,2142,2020,YEM
2143,2143,2020,ZMB


In [18]:
year_code_df[year_code_df["country_code"] == "AFG"]

,level_0,year,country_code
0,0,2010,AFG
195,195,2011,AFG
390,390,2012,AFG
585,585,2013,AFG
780,780,2014,AFG
975,975,2015,AFG
1170,1170,2016,AFG
1365,1365,2017,AFG
1560,1560,2018,AFG
1755,1755,2019,AFG


In [19]:
year_code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   level_0       2145 non-null   int64 
 1   year          2145 non-null   int64 
 2   country_code  2145 non-null   object
dtypes: int64(2), object(1)
memory usage: 50.4+ KB


In [20]:
cols = year_code_df.columns.to_list()

In [21]:
year_code_df[cols[1:]].shape

(2145, 2)

In [26]:
final_year_code_df = pd.merge(year_code_df[cols[1:]], country_code,  how= "left", on = "country_code")[["year", "country_code", "country_name"]].sort_values(by = ["country_code", "year"]).reset_index(drop = True)

In [27]:
final_year_code_df.rename(columns = {"year_x" : "year"}, inplace = True)

In [28]:
path

'/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/curated_datasets'

In [29]:
final_year_code_df.to_csv('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/final_year_code_df.csv', index = False)

In [30]:
final_year_code_df.isna().sum()

year            0
country_code    0
country_name    0
dtype: int64

### NOTE: we must define criteria for filling NaNs in the Type of change dataset, and revolution datasets

In [18]:
#Extract df of type of change alone
final_type_of_change_df = final_year_code_df.merge(revo_raw, how = "left")[["year", "country_code", "country_name", "type_of_change"]]#.isna().sum()

In [19]:
final_type_of_change_df[['type_of_change']] = final_type_of_change_df[['type_of_change']].fillna(value=False)# fill ,missing values with False
final_type_of_change_df[['type_of_change']] = final_type_of_change_df[['type_of_change']].astype(float)# convert values to float
cols_type = final_type_of_change_df.columns.to_list()
cols_type

['year', 'country_code', 'country_name', 'type_of_change']

In [20]:
cols_type[2:0:-1] + cols_type[0:4:3]

['country_name', 'country_code', 'year', 'type_of_change']

In [21]:
final_type_of_change_df = final_type_of_change_df[cols_type[2:0:-1] + cols_type[0:4:3]]
final_type_of_change_df

,country_name,country_code,year,type_of_change
0,Afghanistan,AFG,2010,0.0
1,Afghanistan,AFG,2011,0.0
2,Afghanistan,AFG,2014,0.0
3,Afghanistan,AFG,2015,0.0
4,Afghanistan,AFG,2017,0.0
...,...,...,...,...
2149,Zimbabwe,ZWE,2011,0.0
2150,Zimbabwe,ZWE,2019,0.0
2151,Zimbabwe,ZWE,2010,0.0
2152,Zimbabwe,ZWE,2016,0.0


In [22]:
#Extract df of revolution alone
final_revo_df = final_year_code_df.merge(revo_raw, how = "left")[["year", "country_code", "country_name", "Revolution"]]#.isna().sum()

In [23]:
final_revo_df[['Revolution']] = final_revo_df[['Revolution']].fillna(value=False)# fill ,missing values with False
final_revo_df[['Revolution']] = final_revo_df[['Revolution']].astype(float)# convert values to float
cols_revo = final_revo_df.columns.to_list()

In [24]:
cols_revo[2:0:-1] + cols_revo[0:4:3]

['country_name', 'country_code', 'year', 'Revolution']

In [25]:
final_revo_df = final_revo_df[cols_revo[2:0:-1] + cols_revo[0:4:3]]
final_revo_df

,country_name,country_code,year,Revolution
0,Afghanistan,AFG,2010,0.0
1,Afghanistan,AFG,2011,0.0
2,Afghanistan,AFG,2014,0.0
3,Afghanistan,AFG,2015,0.0
4,Afghanistan,AFG,2017,0.0
...,...,...,...,...
2149,Zimbabwe,ZWE,2011,0.0
2150,Zimbabwe,ZWE,2019,0.0
2151,Zimbabwe,ZWE,2010,0.0
2152,Zimbabwe,ZWE,2016,0.0


### Expor the final datasets

In [26]:
final_revo_df.to_csv(os.path.join(path, "Revolution_final_df.csv"), index = False)
final_type_of_change_df.to_csv(os.path.join(path, "Type_of_cchange_df.csv"), index = False)